### 영수증으로 얻을 수 있는 포인트 알아보기

In [93]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

##### 01. 데이터프레임 생성
- 영수증 전처리

In [94]:
receipt_data = '''
165
(겨
[대
190

이마트 세충점(044)90:
06-06-50913
세종쪽별사치시

밥왼더블:

호 시서자 쑥내8
파프리카(홀합)
강원 애호박

덴)부드러문치즈252
항브로클리(2일/불)
후레쉬센터 흙대파
마칠에주스 포도1.8
풀무원 서주도콩콩
덴)부드러문치즈250
날양 맛있는무류 Ｌ
쿠리쌀 프레이ㅋ490
엘앤멤즈핏1506
'''
receipt = receipt_data.split('\n')
receipt_df = pd.DataFrame(receipt, columns=['title'])
receipt_df['title'] = receipt_df['title'].apply(lambda x: re.sub('[^가-힣\s]', '', x))
receipt_df = receipt_df[receipt_df.title != '']
receipt_df.set_index('title', inplace=True)
receipt_df.reset_index(inplace=True)
receipt_df

,title
0,겨
1,대
2,이마트 세충점
3,세종쪽별사치시
4,밥왼더블
5,호 시서자 쑥내
6,파프리카홀합
7,강원 애호박
8,덴부드러문치즈
9,항브로클리일불


- 리스트 전처리

In [95]:
# 전처리 전 데이터 불러오기
list_df = pd.read_csv('../data/친환경 제품(초안).csv')
# 중복 데이터 제거
list_df.drop_duplicates(subset=['title'], inplace=True)
# 불필요한 문자 제거
list_df['title_mod'] = list_df['title'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
list_df['title_mod'] = list_df['title_mod'].apply(lambda x: re.sub(r'\[[^)]*\]', '', x))
list_df['title_mod'] = list_df['title_mod'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', x))
list_df['point'] = list_df['point'].apply(lambda x: re.sub(r'[^0-9]+', '', x))
list_df['point'] = list_df['point'].astype(int)
# 인덱스 정렬
list_df.set_index('title', inplace=True)
list_df.reset_index(inplace=True)
# 컬럼별 공백 제거
list_df['market'] = list_df['market'].str.lstrip(' ')
list_df['title'] = list_df['title'].str.lstrip(' ')
list_df['title_mod'] = list_df['title_mod'].str.lstrip(' ')
# 빈 문자열 nan값으로 교체 후 삭제
list_df = list_df.replace('', np.nan)
list_df = list_df.dropna(axis=0)
# 최종본으로 저장
list_df.to_csv('../data/eco_product.csv', index=False)

In [96]:
list_df = pd.read_csv('../data/eco_product.csv')
list_df.isna().sum()

title        0
point        0
market       0
title_mod    0
dtype: int64

##### 02. 유사도 검사
- TF-IDF 벡터화

In [97]:
tv = TfidfVectorizer()
receipt_tv = tv.fit_transform(receipt_df['title'])
list_tv = tv.transform(list_df['title_mod'])

- 코사인 유사도 계산

In [98]:
cosine_similarities = cosine_similarity(receipt_tv, list_tv)


- 유사도 컬럼으로 추가

In [99]:
receipt_df['cosine_similarity'] = cosine_similarities.max(axis=1)

- 유사한 항목 추가

In [100]:
most_similar_indices = cosine_similarities.argmax(axis=1)
most_similar_items = list_df.loc[most_similar_indices, 'title_mod'].values
receipt_df['most_similar_item'] = most_similar_items
receipt_df['point'] = list_df.loc[most_similar_indices, 'point'].values
receipt_df['market'] = list_df.loc[most_similar_indices, 'market'].values

In [101]:
receipt_df

,title,cosine_similarity,most_similar_item,point,market
0,겨,0.000000,수실류은행,10,로컬푸드 직매장
1,대,0.000000,수실류은행,10,로컬푸드 직매장
2,이마트 세충점,0.000000,수실류은행,10,로컬푸드 직매장
3,세종쪽별사치시,0.000000,수실류은행,10,로컬푸드 직매장
4,밥왼더블,0.000000,수실류은행,10,로컬푸드 직매장
5,호 시서자 쑥내,0.000000,수실류은행,10,로컬푸드 직매장
6,파프리카홀합,0.000000,수실류은행,10,로컬푸드 직매장
7,강원 애호박,1.000000,친환경 강원 애호박,60,이마트
8,덴부드러문치즈,0.000000,수실류은행,10,로컬푸드 직매장
9,항브로클리일불,0.000000,수실류은행,10,로컬푸드 직매장


In [102]:
# cosine_similarity의 값이 0.5 이상인 경우만 추출
receipt_df = receipt_df[receipt_df['cosine_similarity'] >= 0.5]
receipt_df

,title,cosine_similarity,most_similar_item,point,market
7,강원 애호박,1.000000,친환경 강원 애호박,60,이마트
11,마칠에주스 포도,0.707107,유기농 포도,1270,두레생협
12,풀무원 서주도콩콩,0.707107,풀무원 샘물,1420,풀무원샘물


In [103]:
# 포인트 합계
receipt_df['point'].sum()

2750

In [104]:
###
receipt_list = []
point_list = []
for i in receipt_df.index:
    point_list.append(receipt_df['point'][i])
    receipt_list.append(f'''{receipt_df['title'][i]}({str(receipt_df['point'][i])+'% 적립' if receipt_df['point'][i] == 10 else str(receipt_df['point'][i])+'p'})''')
total_item = '\n'.join(receipt_list)
accu = 0
t_point = 0
for i in point_list:
    if i == 10:
        accu += i
    else:
        t_point += i

if accu == 0:
    total_point = f'\n\n총 {t_point}p'
elif t_point == 0:
    total_point = f'\n\n총 {accu}% 적립'
else:
    total_point = f'\n\n총 {accu}% 적립 + {t_point}p'

total_result = total_item + total_point
print(total_result)

강원 애호박(60p)
마칠에주스 포도(1270p)
풀무원 서주도콩콩(1420p)

총 2750p
